#### Библиотеки

In [3]:
import sys

sys.path.append("..")

import warnings

import pandas as pd
import torch
from torch.nn import functional as F
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoModelForCausalLM
)

import os
import pickle

from warp.utils.data import prepare_warp_dataset
from warp.constants import DATASET_DIR, CONFIG_DIR, MODEL_DIR
from pathlib import Path

warnings.filterwarnings("ignore")

%load_ext autoreload
%cd ..
%autoreload 2

device = "cuda" if torch.cuda.is_available() else "cpu"
device

/home/ivanov.dko/projects/test/rl


'cpu'

На этот раз нам нужны две модели. Одну из них мы предположительно обучили в прошлом ноутбуке. Если по какой-то причине она не лежит в `artifacts/reward_model`, можно скачать какую-то другую

In [4]:
model_name = Path(MODEL_DIR, "reward_model")
# model_name = "lvwerra/distilbert-imdb"

reward_tokenizer = AutoTokenizer.from_pretrained(
    model_name, max_length=512, use_fast=True
)
reward_model = AutoModelForSequenceClassification.from_pretrained(
    model_name
)

Осталось только оценить, насколько модель хороша. Одна у нас уже должна быть с прошлого ноутбука, если нет, то запустим ячейку ниже. В остальном нужно поварьировать параметр и оценить, насколько он вообще на что-то влияет. Я предлагаю взглянуть на $\eta$, потому что у него есть прикольная зависимость от KL

In [ ]:
# default
!poetry run python warp/train_warp.py --config-name warp_config run_name=warp_model trainer.eta=0.5

In [ ]:
# eta=0.1
!poetry run python warp/train_warp.py --config-name warp_config run_name=warp_low_eta trainer.eta=0.1

In [ ]:
# eta=0.9
!poetry run python warp/train_warp.py --config-name warp_config run_name=warp_high_eta trainer.eta=0.9

Теперь измерим средний реворд и дивергенцию предложений. KL между SFT и SFT не имеет смысла, предлагаю это опустить. Ну и в какой-то момент я психанул, потому что сохранялись только веса адаптера, так что веса у меня в пикле. Это некрасиво, конечно. К тому же возник баг, из-за которого не получалось десериализовать модель обратно

In [5]:
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

def get_model(name):
    model_name = Path(MODEL_DIR, name)
    try:
        sft_model = AutoModelForCausalLM.from_pretrained(
        model_name
    )
    except:
        with open(model_name, "rb") as f:
            sft_model = CPU_Unpickler(f).load()
    return sft_model


tokenizer_name = Path(MODEL_DIR, "lvwerra/gpt2-imdb")
sft_tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_name, max_length=512, use_fast=True
)
sft_tokenizer.pad_token = sft_tokenizer.eos_token

Чтобы оценивать результаты, надо бы вообще завести отдельную функцию, стоило сделать сразу

In [6]:
def get_reward(prompt, response):
    return (
        torch.stack(
            [
                reward_model(
                    **reward_tokenizer(
                        input,
                        padding=True,
                        return_tensors="pt",
                    )
                ).logits.detach()
                for input in [prompt, response]
            ]
        )
        .mean(dim=2)
        .softmax(0)[1, :]
    )

In [7]:
from warp.utils.train import policy


def generate_from_prompt(model, prompt=None, input_ids=None):
    if input_ids is None:
        input_ids = sft_tokenizer([prompt], return_tensors="pt")["input_ids"]
    else:
        prompt = sft_tokenizer.batch_decode(input_ids)[0]
    completion_ids = model.generate(
        temperature=0.9,
        do_sample=True,
        pad_token_id=sft_tokenizer.eos_token_id,
        input_ids=input_ids,
    )
    completion = sft_tokenizer.batch_decode(completion_ids)
    return {
        "prompt": prompt,
        "completion": completion[0],
        "logps": policy(model, completion_ids, len_generated=10)[0],
        "reward": get_reward(prompt, completion[0])[0]
    }

Датасет у нас фактически тот же, так что можно украсть код у себя самого

In [8]:
import polars as pl

test = load_dataset("imdb", split="test").to_pandas()
test = prepare_warp_dataset(
    pl.DataFrame(test).sample(100).to_dict(as_series=False),
    tokenizer=sft_tokenizer,
)

2024-08-06 02:14:55.075 | INFO     | warp.utils.data:prepare_warp_dataset:164 - Starting tokenizing `text`


In [40]:
test

Dataset({
    features: ['input_ids', 'attention_mask', 'text'],
    num_rows: 100
})

Теперь осталось прогнать на нём каждую модель и посчитать KL + Reward. Reward буду точно так же считать, сравнивая prompt и completion

Веса использованных моделей лежат [тут](https://drive.google.com/file/d/1H5ppIlJP4M2Fy3Dl1F85ERheSaJD1tFH/view?usp=sharing), [тут](https://drive.google.com/file/d/1-7A5sObWbIGyZjSxQYv-ZD_E1lzH7OjK/view?usp=drive_link) и [тут](https://drive.google.com/file/d/1-5MLpNCg2_xb3H_y5v8PGk_ESv-0qE4D/view?usp=sharing)

In [108]:
model = get_model("warp_low_eta.pkl")

In [ ]:
from warp.utils.misc import seed_everything

seed_everything(42)

answers = []
for _ in range(5): # чтоб выборка была побольше
    for key in [
        "lvwerra/gpt2-imdb",
        "warp_model.pkl",
        "warp_low_eta.pkl",
        "warp_high_eta.pkl",
    ]:
        model = get_model(key)
        sub_df = test.map(
            lambda x: generate_from_prompt(
                model, input_ids=x["input_ids"].unsqueeze(0)
            )
        ).to_pandas()
        sub_df["model"] = key
        answers.append(pl.DataFrame(sub_df))

Теперь осталось это нормально изобразить. Как обычно, придётся немного повозиться

In [11]:
aggregated = (
    pl.concat(answers).group_by("model").agg(pl.col("logps", "reward").mean())
).sort("model")
sft_kl_div = aggregated.filter(pl.col("model") == "lvwerra/gpt2-imdb")["logps"][0]
all_stats = aggregated.with_columns(
    kl_div=pl.col("logps") - sft_kl_div, eta=pl.Series([0.0, 0.9, 0.1, 0.5])
)
all_stats

model,logps,reward,kl_div,eta
str,f64,f64,f64,f64
"""lvwerra/gpt2-imdb""",-30.611758,0.487238,0.0,0.0
"""warp_high_eta.pkl""",-30.888689,0.503654,-0.276932,0.9
"""warp_low_eta.pkl""",-30.540535,0.494633,0.071223,0.1
"""warp_model.pkl""",-30.468174,0.509606,0.143584,0.5


У пытливого читателя может возникнуть вопрос - как KL может быть меньше 0. Ответ - я не знаю, возможно здесь и кроется моя ошибка. Но меня и в самой статье смутило, что KL не в совсем привычном мне виде. Там нет суммы, там нет вероятности, это всё очень странно, но по тому, что я гуглил, ошибки там нет. Но может быть есть у меня. Я предлагаю немножко на это забить, потому что вопрос "а между чем вообще считать KL" в любом случае останется без ответа, но да, я вижу проблему

In [41]:
all_stats.plot.scatter(
    "kl_div",
    "reward",
    color="eta",
    title="KL-Eta relation",
    grid=True,
    colorbar=True,
    clabel="eta"
)

:Scatter   [kl_div]   (reward,eta)

По итогу всё равно к сожалению получилось что-то не то. Должно было быть вот как:
1. $\eta = 0$ означает, что модель фактически не обновляется никак и сохраняет то же распределение, что и было, то есть KL практически нулевой, но и награды мы никакой не получаем, потому что не обучаемся толком. У меня награда действительно маленькая, ниже всех, тут успех. С дивергенцией - скорее всего косяк, хотя по модулю она действительно тоже меньше всех
2. Наоборот означает, что мы обучаемся со всей дури, и распределение меняется тоже очень сильно. Если KL по модулю действительно больше всех, тут ещё ладно, то reward, к сожалению, не наибольший. Возможно я не совсем верно его замеряю, либо же не совсем честно его проверяю. Мне кажется, что тестовых примеров маловато, плюс генерации сами по себе подвержены рандому, тут нужно немножко похитрее это сделать, в идеале с какими-то CI

А остальные параметры уже толком и не проверю. Я довольно много игрался с $I$ - числом итераций, но там разумно предположить, что чем их меньше, тем хуже модель обучается - меньше KL, меньше reward. И должно это быть верно для любого гиперпараметры с итерациями - по картинке парето фронт просто съезжает вверх

Так что итог: что-то конечно получилось, но что?
Напоследок хотя бы примеры генераций

In [32]:
pl.concat(answers).filter(pl.col("model") == "warp_high_eta.pkl").select(
    "prompt", "completion"
).sample(5).to_numpy()

array([['What has Rajiv Rai done to himself? Once a hit director of films',
        'What has Rajiv Rai done to himself? Once a hit director of films like this has done it'],
       ["I'm not from USA I'm from central Europe and i think the show",
        "I'm not from USA I'm from central Europe and i think the show is stupid. The characters"],
       ['A female ex-cop who was drummed out of the force for reck',
        'A female ex-cop who was drummed out of the force for recklessly killing a cop is'],
       ['Oh, man, I hated this movie. Granted, the site locations were',
        'Oh, man, I hated this movie. Granted, the site locations were the only "place to'],
       ['Set in Paris in the year 1910, a retired old rich opera singer decides',
        'Set in Paris in the year 1910, a retired old rich opera singer decides to escape his old life']],
      dtype=object)

Негатив там конечно есть, но наверное не так много